In [1]:
#setup, importing modules
import os
os.system("sudo apt-get update")
os.system("sudo apt-get install -y python3-opencv")
os.system("pip install opencv-python")
os.system("pip install matplotlib")
os.system("pip install pandas")
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
python3-opencv is already the newest version (4.5.4+dfsg-9ubuntu4).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


/tmp/ipykernel_43529/1653740564.py:11: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [18]:
#functions
def normit(image):

    """ function for reading in an image file, taking it's color histograms and normailizing them. """
    hist = cv2.calcHist([image],[0,1,2],None,[255,255,255],[0,256,0,256,0,256])
    normalized_hist = cv2.normalize(hist, hist, 0, 1.0, cv2.NORM_MINMAX)
    return(normalized_hist)

def count_hist_dist(filename,name_list,image_list,n_returns):

    """function for finding Distance from a set image file's color histograms.
    filename: the name of the file, which will be the target
    name_list: a list of all filenames the comparison should include
    image_list: a list of images. the images are loaded in, with three colour channels and two spacial
    dimensions. The list of images should have the same length and order as the list of names.
    n_returns: (int) how many filenames and distances should be returned."""

    #find target according file index
    target_ind = name_list.index(filename) 
    #create target comparison hist
    target = normit(image_list[target_ind])

    #compare to target
    comp_list =[]
    for i in image_list:
        norm =  normit(i)
        histcomp = round(cv2.compareHist(target,norm,cv2.HISTCMP_CHISQR), 2)
        #save comparison values
        comp_list.append(histcomp)

    #assign filenames to comparison values

    df_pd = pd.DataFrame({"Filename":name_list})
    df_pd["Distance"] = comp_list

    #sort and filter
    #sort ascending
    df_sorted = df_pd.sort_values(axis=0, by= "Distance", ascending= True)
    #drops 1st entry (it's the target image themselves)
    df_sorted = df_sorted.query('Distance != 0')
    #keeps 5 closest images
    df_fin = df_sorted[:n_returns]

    return(df_fin)


In [5]:
#load in files
# loop load all flowers and names and save it
fp = os.path.join("..","in")
fp_s = sorted(os.listdir(fp))


image_list = []
name_list = []
for i in fp_s:
    filepath = os.path.join(fp,i)
    image = cv2.imread(filepath)
    names = i
    name_list.append(i)
    image_list.append(image)

In [25]:
#Find distances for e.g. target file "image_0121.jpg"

target_name = "image_0121.jpg"
count_df = count_hist_dist(target_name,name_list,image_list,5)

In [26]:
#save output
out_p = os.path.join("..","out",target_name + "_comp.csv")
count_df.to_csv(out_p, index=False)